# Creación de dataset para modelado del IDH a partir del gasto municipal

Este proyecto se enfoca en la creación de un conjunto de datos con el que se pueda modelar el Indice de Desarrollo Humano a partir de los Capítulos del gasto municipal en México en 2015.

Para ello se conjugan 3 conjuntos de datos obtenidos de los portales de datos abiertos del gobierno de México. El primero de ellos es el conjunto de datos de las finanzas municipales, el sgundo, el catálogo de muncipios de México y el tercero, el conjunto del Indice de Desarrallo Humano por municipio.

Se utilizan las librearías "Numpy" y "Pandas" las cuales se ocupan en la carga, limpieza y transformación de los datos, lo cual es el kid de este proyecto.

Al final se obtiene un dataset o dataframe con una "muestra representativa" de 420 municipios, de los más de 2400 que existen en México. Los datos están organizados en forma tabular con 22 columnas, entre las variables sobresalientes se encuentran: nombre de municipio, id de municipio, idh, clasificación ordinal, clasificación categórica, total de egresos, capítulos de egreso y porcentaje de los capítulos de egreso respecto al gasto total.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
gm15 = pd.read_csv("C:/Users/PC/Documents/Investigacion/idh_gasto_municipal/efipem_municipal_anual_tr_cifra_2015.csv")

In [3]:
cat_munic = pd.read_csv("C:/Users/PC/Documents/Investigacion/idh_gasto_municipal/tc_municipio.csv")

In [4]:
idh15 = pd.read_csv("C:/Users/PC/Documents/Investigacion/idh_gasto_municipal/idh_15.csv", encoding = "iso8859_2")

## Selección de los municipios a estudiar

Se seleccionará una muestra representativa de los municipios a estudiar, los cuales son 2454.

In [5]:
cat_munic.head()

,ID_ENTIDAD,ID_MUNICIPIO,NOM_MUN
0,1,1,Aguascalientes
1,1,2,Asientos
2,1,3,Calvillo
3,1,4,Cosío
4,1,5,Jesús María


In [6]:
cat_munic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2454 entries, 0 to 2453
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID_ENTIDAD    2454 non-null   int64 
 1   ID_MUNICIPIO  2454 non-null   int64 
 2   NOM_MUN       2454 non-null   object
dtypes: int64(2), object(1)
memory usage: 57.6+ KB


## Fórmula para la muestra

La fórmula para calcular una muestra estadística representativa es la siguiente:
                       $n = (N*(z^2)*p*(1-p))/((e^2)*(N-1)+(z^2)*p*(1-p))$
              
*n es el tamaño de la muestra.

*N es el tamaño de la población.

*z es el valor crítico de la distribución normal estándar para un nivel de confianza determinado. Por ejemplo, para un nivel de confianza del 90%, z = 1.65, para un nivel de confianza de 95%, z = 1.96 y para un 99% de confianza, z = 2.58.

*p es la proporción de la población que se espera que tenga una característica determinada.

*e es el margen de error deseado.
             

In [7]:
# Fórmula para calcular una muestra estadística representativa
N = 2454
z = 1.96
p = 0.5
e = 0.05
n = (𝑁*(𝑧**2)*𝑝*(1-𝑝))/((𝑒**2)*(𝑁-1)+(𝑧**2)*𝑝*(1-𝑝))
n

332.2789832085606

Aunque el número mínimo para que la muestra sea representativa es de 332 (aproximadamente), se selecciona una muestra de 500 municipios a fin de poder particionarla si el modelo o método de análisis lo requiere.

In [8]:
# Se utiliza el método "sample" para seleccionar 500 municipios en un subdataframe a partir del df de catálogo municipios.
muestra_munic = cat_munic.sample(500, random_state=1235)
muestra_munic[0:10]

,ID_ENTIDAD,ID_MUNICIPIO,NOM_MUN
1291,20,298,San Pablo Villa de Mitla
1475,20,482,Santiago Pinotepa Nacional
408,12,50,Pungarabato
1004,20,11,Calihualá
427,12,69,Xalpatláhuac
2036,29,19,Tepetitla de Lardizábal
1869,25,2,Angostura
198,7,117,Montecristo de Guerrero
388,12,30,Florencio Villarreal
1529,20,536,San Vicente Nuñú


In [9]:
# Verificamos el subdataframe de la muestra de municipios
muestra_munic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 1291 to 1711
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID_ENTIDAD    500 non-null    int64 
 1   ID_MUNICIPIO  500 non-null    int64 
 2   NOM_MUN       500 non-null    object
dtypes: int64(2), object(1)
memory usage: 15.6+ KB


## Seleccionamos los municipios de la muestra en el df de IDH

In [10]:
# Observamos el df de idh
idh15.head()

,estado,municipio,cve_inegi,id_estado,id_municipio,clasif,idh,gdh,tmi,ipca,isal,ieduc,iing
0,Aguascalientes,Aguascalientes,1001,1,1,52,0.804,MUY ALTO,5.8,3498.3,0.965,0.671,0.804
1,Aguascalientes,Asientos,1002,1,2,1025,0.665,MEDIO,19.4,1427.5,0.85,0.517,0.669
2,Aguascalientes,Calvillo,1003,1,3,1032,0.664,MEDIO,17.6,1506.4,0.865,0.5,0.677
3,Aguascalientes,Cosío,1004,1,4,660,0.699,MEDIO,18.3,1835.8,0.859,0.562,0.707
4,Aguascalientes,Jesús María,1005,1,5,297,0.742,ALTO,12.2,2611.8,0.911,0.59,0.76


In [11]:
# Identificamos los valores no válidos, en este caso, no disponibles.
idh15[idh15["idh"] == "ND"]

,estado,municipio,cve_inegi,id_estado,id_municipio,clasif,idh,gdh,tmi,ipca,isal,ieduc,iing
1811,Quintana Roo,Bacalar,23010,23,10,ND,ND,Bajo,ND,ND,ND,ND,ND


In [12]:
idh15.dtypes

estado          object
municipio       object
cve_inegi        int64
id_estado        int64
id_municipio     int64
clasif          object
idh             object
gdh             object
tmi             object
ipca            object
isal            object
ieduc           object
iing            object
dtype: object

In [13]:
# Eliminamos el registro que tiene datos faltantes.
idh15.drop(idh15[idh15["clasif"] == "ND"].index, inplace = True)

In [14]:
# Verificamos la eliminación de la fila
idh15[idh15["idh"] == "ND"]

,estado,municipio,cve_inegi,id_estado,id_municipio,clasif,idh,gdh,tmi,ipca,isal,ieduc,iing


In [15]:
# Verificamos que no se haya eliminado alguna columna
idh15.head()

,estado,municipio,cve_inegi,id_estado,id_municipio,clasif,idh,gdh,tmi,ipca,isal,ieduc,iing
0,Aguascalientes,Aguascalientes,1001,1,1,52,0.804,MUY ALTO,5.8,3498.3,0.965,0.671,0.804
1,Aguascalientes,Asientos,1002,1,2,1025,0.665,MEDIO,19.4,1427.5,0.85,0.517,0.669
2,Aguascalientes,Calvillo,1003,1,3,1032,0.664,MEDIO,17.6,1506.4,0.865,0.5,0.677
3,Aguascalientes,Cosío,1004,1,4,660,0.699,MEDIO,18.3,1835.8,0.859,0.562,0.707
4,Aguascalientes,Jesús María,1005,1,5,297,0.742,ALTO,12.2,2611.8,0.911,0.59,0.76


In [16]:
# Concatenamos los dos DataFrames, con la función "merge" (idh15 y muestra_munic) a fin de generar uno nuevo con los datos donde ambos coinciden.
df_merged = pd.merge(left=idh15, right= muestra_munic, how = "inner", left_on = ["id_estado", "id_municipio"], right_on = ["ID_ENTIDAD", "ID_MUNICIPIO"])

In [17]:
df_merged.head()

,estado,municipio,cve_inegi,id_estado,id_municipio,clasif,idh,gdh,tmi,ipca,isal,ieduc,iing,ID_ENTIDAD,ID_MUNICIPIO,NOM_MUN
0,Aguascalientes,Asientos,1002,1,2,1025,0.665,MEDIO,19.4,1427.5,0.85,0.517,0.669,1,2,Asientos
1,Aguascalientes,Cosío,1004,1,4,660,0.699,MEDIO,18.3,1835.8,0.859,0.562,0.707,1,4,Cosío
2,Aguascalientes,San José de Gracia,1008,1,8,576,0.708,ALTO,14.3,1701.4,0.893,0.571,0.695,1,8,San José de Gracia
3,Baja California,Ensenada,2001,2,1,211,0.755,ALTO,17.7,3349,0.864,0.625,0.798,2,1,Ensenada
4,Baja California,Tijuana,2004,2,4,90,0.786,ALTO,10.3,3577.1,0.927,0.647,0.808,2,4,Tijuana


In [18]:
# Obtenemos información de la estructura del nuevo dataframe
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 495
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   estado        496 non-null    object
 1   municipio     496 non-null    object
 2   cve_inegi     496 non-null    int64 
 3   id_estado     496 non-null    int64 
 4   id_municipio  496 non-null    int64 
 5   clasif        496 non-null    object
 6   idh           496 non-null    object
 7   gdh           496 non-null    object
 8   tmi           496 non-null    object
 9   ipca          496 non-null    object
 10  isal          496 non-null    object
 11  ieduc         496 non-null    object
 12  iing          496 non-null    object
 13  ID_ENTIDAD    496 non-null    int64 
 14  ID_MUNICIPIO  496 non-null    int64 
 15  NOM_MUN       496 non-null    object
dtypes: int64(5), object(11)
memory usage: 65.9+ KB


In [19]:
# Eliminamos algunas columnas que no utilizaremos ("tmi", "ipca", "isal", "ieduc" y "iing")
df_merged = df_merged.drop(["tmi", "ipca", "isal", "ieduc", "iing"], axis = 1)
df_merged.head()

,estado,municipio,cve_inegi,id_estado,id_municipio,clasif,idh,gdh,ID_ENTIDAD,ID_MUNICIPIO,NOM_MUN
0,Aguascalientes,Asientos,1002,1,2,1025,0.665,MEDIO,1,2,Asientos
1,Aguascalientes,Cosío,1004,1,4,660,0.699,MEDIO,1,4,Cosío
2,Aguascalientes,San José de Gracia,1008,1,8,576,0.708,ALTO,1,8,San José de Gracia
3,Baja California,Ensenada,2001,2,1,211,0.755,ALTO,2,1,Ensenada
4,Baja California,Tijuana,2004,2,4,90,0.786,ALTO,2,4,Tijuana


## Conjunto de finanzas municipales
En la siguiente celda se muestra la vista general del dataset de finanzas municipales, el cual requiere una transformación para eliminar ciertas columnas, además de que los capítulos del gasto se coloquen como columnas y los valores como filas.

In [20]:
gm15.head()

,PROD_EST,COBERTURA,ANIO,ID_ENTIDAD,ID_MUNICIPIO,TEMA,CATEGORIA,DESCRIPCION_CATEGORIA,VALOR,ESTATUS
0,Estadística de Finanzas Públicas Estatales y M...,Municipal,2015,1,1,Egresos,Tema,Total de egresos,3247148647,Cifras Definitivas.
1,Estadística de Finanzas Públicas Estatales y M...,Municipal,2015,1,1,Egresos,Capítulo,Servicios personales,1293717343,Cifras Definitivas.
2,Estadística de Finanzas Públicas Estatales y M...,Municipal,2015,1,1,Egresos,Concepto,Remuneraciones al personal,400461357,Cifras Definitivas.
3,Estadística de Finanzas Públicas Estatales y M...,Municipal,2015,1,1,Egresos,Partida Genérica,Dependencias diversas,400461357,Cifras Definitivas.
4,Estadística de Finanzas Públicas Estatales y M...,Municipal,2015,1,1,Egresos,Concepto,Remuneraciones adicionales y especiales,370636723,Cifras Definitivas.


In [21]:
# Aislamos los "id_estado" e "id_municipio" para generar un nuevo dataframe de las finanzas de estos municipios
df_ids = df_merged[["id_estado", "id_municipio"]]
df_ids.head()

,id_estado,id_municipio
0,1,2
1,1,4
2,1,8
3,2,1
4,2,4


## Función finanzas_munic
La siguiente función toma de cada municipio el "id" e "id entidad", con ellos selecciona a dicho municipio en el conjunto de
finanzas municipales, selecciona el tema de egresos, de los mismos selecciona los capítulos del gasto (excepto disponibilidad final) y gasto total, elimina las columnas innecesarias y genera una tabla dinámica de cada municipio, transponiendo categorías y valores de forma conveniente.

In [22]:
def finanzas_munic(id_ent, id_munic):
    # Seleccionamos el municipio por su "id_entidad" y id_municipio
    fin_munic = gm15[(gm15["ID_ENTIDAD"] == id_ent) & (gm15["ID_MUNICIPIO"] == id_munic)]
    # Seleccionamos el apartado de egresos
    fin_munic = fin_munic[fin_munic["TEMA"] == "Egresos"]
    # Seleccionamos el total de egresos y lo guardamos en un df
    df_total_egresos = fin_munic[fin_munic["DESCRIPCION_CATEGORIA"] == "Total de egresos"]
    # Seleccionamos los egresos
    egresos = fin_munic[fin_munic["TEMA"] == "Egresos"]
    # Seleccionamos los capítulos del gasto
    egresos = egresos[(egresos["CATEGORIA"] == "Capítulo")]
    # Seleccionamos de la columna "DESCRIPCION_CATEGORIA" los valores diferentes de "Disponibilidad final".
    egresos = egresos[egresos["DESCRIPCION_CATEGORIA"] != "Disponibilidad final"]
    # Concatenamos los df´s de "total_egresos" y "egresos"
    egresos = pd.concat([df_total_egresos, egresos], axis = 0)
    # Con la siguiente línea seleccionamos las columnas que nos interesan.
    egresos = egresos[["ID_ENTIDAD", "ID_MUNICIPIO", "CATEGORIA", "DESCRIPCION_CATEGORIA", "VALOR"]]
    # Realizamos una pivot_table a fin de organizar las columnas y valores
    egr_pivot = pd.pivot_table(egresos, index = ["ID_ENTIDAD", "ID_MUNICIPIO"], columns = "DESCRIPCION_CATEGORIA", 
                           values = "VALOR", sort = False)
    return egr_pivot

## Función principal
Esta función inicia con un dataframe vacío, el cual será actualizado en cada iteración de un bucle que recorre los "n" id's de los municipios seleccionados en la muestra, los cuales suministra a la función "finanzas_munic" (dentro del bucle), para generar la tabla dinámica de cada municipio, al final del bucle se concatena la "iesima" tabla dinámica y/o se actualiza el dataframe vacío. Esta función ofrece como resultado la concatenación de las "n" tablas dinámicas de los municipios y sus capítulos de egreso como variables.

In [23]:
def principal(df_ids):
    # Definimos un dataframe vacío el cual se irá concatenando para formar el df final
    df_concat = pd.DataFrame()
    
    for i in range(0, len(df_ids["id_estado"]), 1):
        id_ent = df_ids["id_estado"][i]
        id_munic = df_ids["id_municipio"][i]
        
        # Llamamos a la función "finanzas_munic()"
        egr_pivot = finanzas_munic(id_ent, id_munic)
        
        # Llamamos a la función "concatenar_dfs()"
        df_concat = pd.concat([df_concat, egr_pivot], axis = 0)
        
    return df_concat

In [24]:
df1 = principal(df_ids)

In [25]:
type(df1)

pandas.core.frame.DataFrame

In [26]:
# Observamos el resultado de la transformación del conjunto de finanzas municipales.
df1.head()

DESCRIPCION_CATEGORIA    Total de egresos  Servicios personales  \
ID_ENTIDAD ID_MUNICIPIO                                           
1          2                    152726780          6.108078e+07   
           4                    102713281          4.573242e+07   
           8                    101257782          4.102323e+07   
2          1                   1652675878          9.578552e+08   
           4                   5730603903          3.408784e+09   

DESCRIPCION_CATEGORIA    Materiales y suministros  Servicios generales  \
ID_ENTIDAD ID_MUNICIPIO                                                  
1          2                              9491404             18743397   
           4                              6171900              7491884   
           8                              5016478             13176666   
2          1                            149529041            215489829   
           4                            497731034            517665436   

DESCRIPCION_CATEGORIA    Transferencias, asignaciones, subsidios y otras ayudas  \
ID_ENTIDAD ID_MUNICIPIO                                                           
1          2                                                    12990534.0        
           4                                                     7236061.0        
           8                                                    11482239.0        
2          1                                                    93697975.0        
           4                                                   375628846.0        

DESCRIPCION_CATEGORIA    Bienes muebles, inmuebles e intangibles  \
ID_ENTIDAD ID_MUNICIPIO                                            
1          2                                            120474.0   
           4                                           1595001.0   
           8                                           5289744.0   
2          1                                          56198747.0   
           4                                         180836183.0   

DESCRIPCION_CATEGORIA    Inversión pública  Deuda pública  \
ID_ENTIDAD ID_MUNICIPIO                                     
1          2                    34970738.0      4003443.0   
           4                    34486010.0            NaN   
           8                    23253517.0            NaN   
2          1                   127376775.0     52528341.0   
           4                   608300478.0    141658112.0   

DESCRIPCION_CATEGORIA    Inversiones financieras y otras provisiones  \
ID_ENTIDAD ID_MUNICIPIO                                                
1          2                                                     NaN   
           4                                                     NaN   
           8                                                     NaN   
2          1                                                     NaN   
           4                                                     NaN   

DESCRIPCION_CATEGORIA    Otros egresos  
ID_ENTIDAD ID_MUNICIPIO                 
1          2                       NaN  
           4                       NaN  
           8                       NaN  
2          1                       NaN  
           4                       NaN

In [27]:
pd.__version__

'1.5.2'

In [28]:
# Eliminamos las últimas 2 columnas
df1 = df1.drop(["Inversiones financieras y otras provisiones", "Otros egresos"], axis = 1)
df1.head()

DESCRIPCION_CATEGORIA    Total de egresos  Servicios personales  \
ID_ENTIDAD ID_MUNICIPIO                                           
1          2                    152726780          6.108078e+07   
           4                    102713281          4.573242e+07   
           8                    101257782          4.102323e+07   
2          1                   1652675878          9.578552e+08   
           4                   5730603903          3.408784e+09   

DESCRIPCION_CATEGORIA    Materiales y suministros  Servicios generales  \
ID_ENTIDAD ID_MUNICIPIO                                                  
1          2                              9491404             18743397   
           4                              6171900              7491884   
           8                              5016478             13176666   
2          1                            149529041            215489829   
           4                            497731034            517665436   

DESCRIPCION_CATEGORIA    Transferencias, asignaciones, subsidios y otras ayudas  \
ID_ENTIDAD ID_MUNICIPIO                                                           
1          2                                                    12990534.0        
           4                                                     7236061.0        
           8                                                    11482239.0        
2          1                                                    93697975.0        
           4                                                   375628846.0        

DESCRIPCION_CATEGORIA    Bienes muebles, inmuebles e intangibles  \
ID_ENTIDAD ID_MUNICIPIO                                            
1          2                                            120474.0   
           4                                           1595001.0   
           8                                           5289744.0   
2          1                                          56198747.0   
           4                                         180836183.0   

DESCRIPCION_CATEGORIA    Inversión pública  Deuda pública  
ID_ENTIDAD ID_MUNICIPIO                                    
1          2                    34970738.0      4003443.0  
           4                    34486010.0            NaN  
           8                    23253517.0            NaN  
2          1                   127376775.0     52528341.0  
           4                   608300478.0    141658112.0

In [29]:
# Observamos los datos nulos o faltantes en el dataset resultante
df1.isnull().sum()

DESCRIPCION_CATEGORIA
Total de egresos                                            0
Servicios personales                                        1
Materiales y suministros                                    0
Servicios generales                                         0
Transferencias, asignaciones, subsidios y otras ayudas      6
Bienes muebles, inmuebles e intangibles                    52
Inversión pública                                          12
Deuda pública                                             245
dtype: int64

En la celda anterior las últimas tres columnas presentan datos faltanes, la segunda muestra un dato nulo o no disponible.

In [30]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 421 entries, (1, 2) to (32, 56)
Data columns (total 8 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Total de egresos                                        421 non-null    int64  
 1   Servicios personales                                    420 non-null    float64
 2   Materiales y suministros                                421 non-null    int64  
 3   Servicios generales                                     421 non-null    int64  
 4   Transferencias, asignaciones, subsidios y otras ayudas  415 non-null    float64
 5   Bienes muebles, inmuebles e intangibles                 369 non-null    float64
 6   Inversión pública                                       409 non-null    float64
 7   Deuda pública                                           176 non-null    float64
dtypes: float64(5), int64(3)
memory 

In [31]:
# Se exporta el dataframe a csv para que sea más fácil manejarlo, ya que actualmente se encuetra como tabla dinámica.
df1.to_csv(r"C:\Users\PC\Documents\Investigacion\idh_gasto_municipal\df1.csv")

In [32]:
# Cargamos el dataset exportado
df2 = pd.read_csv(r"C:\Users\PC\Documents\Investigacion\idh_gasto_municipal\df1.csv")

In [33]:
# Observamos las primeras filas y columnas del conjunto
df2.head()

,ID_ENTIDAD,ID_MUNICIPIO,Total de egresos,Servicios personales,Materiales y suministros,Servicios generales,"Transferencias, asignaciones, subsidios y otras ayudas","Bienes muebles, inmuebles e intangibles",Inversión pública,Deuda pública
0,1,2,152726780,6.108078e+07,9491404,18743397,12990534.0,120474.0,34970738.0,4003443.0
1,1,4,102713281,4.573242e+07,6171900,7491884,7236061.0,1595001.0,34486010.0,NaN
2,1,8,101257782,4.102323e+07,5016478,13176666,11482239.0,5289744.0,23253517.0,NaN
3,2,1,1652675878,9.578552e+08,149529041,215489829,93697975.0,56198747.0,127376775.0,52528341.0
4,2,4,5730603903,3.408784e+09,497731034,517665436,375628846.0,180836183.0,608300478.0,141658112.0


In [34]:
df2 = pd.DataFrame(df2)

In [35]:
type(df2)

pandas.core.frame.DataFrame

In [36]:
# Verificamos el tipo de dato de cada columna
df2.dtypes

ID_ENTIDAD                                                  int64
ID_MUNICIPIO                                                int64
Total de egresos                                            int64
Servicios personales                                      float64
Materiales y suministros                                    int64
Servicios generales                                         int64
Transferencias, asignaciones, subsidios y otras ayudas    float64
Bienes muebles, inmuebles e intangibles                   float64
Inversión pública                                         float64
Deuda pública                                             float64
dtype: object

In [37]:
# Observamos los "NaN" o datos nulos.
df2.isna().sum()

ID_ENTIDAD                                                  0
ID_MUNICIPIO                                                0
Total de egresos                                            0
Servicios personales                                        1
Materiales y suministros                                    0
Servicios generales                                         0
Transferencias, asignaciones, subsidios y otras ayudas      6
Bienes muebles, inmuebles e intangibles                    52
Inversión pública                                          12
Deuda pública                                             245
dtype: int64

In [38]:
# Rellenamos los datos faltantes de la columna "Deuda pública" con cero = 0
df2["Deuda pública"] = df2["Deuda pública"].fillna(0)

In [39]:
# Verificamos el rellenado
df2.isna().sum()

ID_ENTIDAD                                                 0
ID_MUNICIPIO                                               0
Total de egresos                                           0
Servicios personales                                       1
Materiales y suministros                                   0
Servicios generales                                        0
Transferencias, asignaciones, subsidios y otras ayudas     6
Bienes muebles, inmuebles e intangibles                   52
Inversión pública                                         12
Deuda pública                                              0
dtype: int64

In [40]:
# Rellenamos otras columnas 
df2["Inversión pública"] = df2["Inversión pública"].fillna(0)
df2["Bienes muebles, inmuebles e intangibles"] = df2["Bienes muebles, inmuebles e intangibles"].fillna(0)
df2["Transferencias, asignaciones, subsidios y otras ayudas"] = df2["Transferencias, asignaciones, subsidios y otras ayudas"].fillna(0)

In [41]:
# Verificamos
df2.isna().sum()

ID_ENTIDAD                                                0
ID_MUNICIPIO                                              0
Total de egresos                                          0
Servicios personales                                      1
Materiales y suministros                                  0
Servicios generales                                       0
Transferencias, asignaciones, subsidios y otras ayudas    0
Bienes muebles, inmuebles e intangibles                   0
Inversión pública                                         0
Deuda pública                                             0
dtype: int64

In [42]:
# Se elimina la fila restante que contiene un valor nulo
df2.dropna(inplace = True)

In [43]:
# Estructura del dataset final despues de eliminar filas no válidas y/o inconsistentes
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 0 to 420
Data columns (total 10 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ID_ENTIDAD                                              420 non-null    int64  
 1   ID_MUNICIPIO                                            420 non-null    int64  
 2   Total de egresos                                        420 non-null    int64  
 3   Servicios personales                                    420 non-null    float64
 4   Materiales y suministros                                420 non-null    int64  
 5   Servicios generales                                     420 non-null    int64  
 6   Transferencias, asignaciones, subsidios y otras ayudas  420 non-null    float64
 7   Bienes muebles, inmuebles e intangibles                 420 non-null    float64
 8   Inversión pública                       

In [44]:
# Verificamos la transformación
df2.head()

,ID_ENTIDAD,ID_MUNICIPIO,Total de egresos,Servicios personales,Materiales y suministros,Servicios generales,"Transferencias, asignaciones, subsidios y otras ayudas","Bienes muebles, inmuebles e intangibles",Inversión pública,Deuda pública
0,1,2,152726780,6.108078e+07,9491404,18743397,12990534.0,120474.0,34970738.0,4003443.0
1,1,4,102713281,4.573242e+07,6171900,7491884,7236061.0,1595001.0,34486010.0,0.0
2,1,8,101257782,4.102323e+07,5016478,13176666,11482239.0,5289744.0,23253517.0,0.0
3,2,1,1652675878,9.578552e+08,149529041,215489829,93697975.0,56198747.0,127376775.0,52528341.0
4,2,4,5730603903,3.408784e+09,497731034,517665436,375628846.0,180836183.0,608300478.0,141658112.0


In [45]:
# Observamos el tipo de dato de cada columna
df2.dtypes

ID_ENTIDAD                                                  int64
ID_MUNICIPIO                                                int64
Total de egresos                                            int64
Servicios personales                                      float64
Materiales y suministros                                    int64
Servicios generales                                         int64
Transferencias, asignaciones, subsidios y otras ayudas    float64
Bienes muebles, inmuebles e intangibles                   float64
Inversión pública                                         float64
Deuda pública                                             float64
dtype: object

In [46]:
# Generamos una nueva columna para el df2, la cual será el porcentaje de "servicios personales" respecto del "total de egresos".
df2["sp_porc"] = df2["Servicios personales"] / df2["Total de egresos"]

In [47]:
df2.head(3)

,ID_ENTIDAD,ID_MUNICIPIO,Total de egresos,Servicios personales,Materiales y suministros,Servicios generales,"Transferencias, asignaciones, subsidios y otras ayudas","Bienes muebles, inmuebles e intangibles",Inversión pública,Deuda pública,sp_porc
0,1,2,152726780,61080777.0,9491404,18743397,12990534.0,120474.0,34970738.0,4003443.0,0.399935
1,1,4,102713281,45732425.0,6171900,7491884,7236061.0,1595001.0,34486010.0,0.0,0.445244
2,1,8,101257782,41023229.0,5016478,13176666,11482239.0,5289744.0,23253517.0,0.0,0.405137


In [48]:
# Generamos los porcentajes de las columnas restantes.
df2["ms_porc"] = df2["Materiales y suministros"] / df2["Total de egresos"]
df2["sg_porc"] = df2["Servicios generales"] / df2["Total de egresos"]
df2["taoa_porc"] = df2["Transferencias, asignaciones, subsidios y otras ayudas"] / df2["Total de egresos"]
df2["bmii_porc"] = df2["Bienes muebles, inmuebles e intangibles"] / df2["Total de egresos"]
df2["ip_porc"] = df2["Inversión pública"] / df2["Total de egresos"]
df2["dp_porc"] = df2["Deuda pública"] / df2["Total de egresos"]

In [49]:
df2.head(3)

,ID_ENTIDAD,ID_MUNICIPIO,Total de egresos,Servicios personales,Materiales y suministros,Servicios generales,"Transferencias, asignaciones, subsidios y otras ayudas","Bienes muebles, inmuebles e intangibles",Inversión pública,Deuda pública,sp_porc,ms_porc,sg_porc,taoa_porc,bmii_porc,ip_porc,dp_porc
0,1,2,152726780,61080777.0,9491404,18743397,12990534.0,120474.0,34970738.0,4003443.0,0.399935,0.062146,0.122725,0.085057,0.000789,0.228976,0.026213
1,1,4,102713281,45732425.0,6171900,7491884,7236061.0,1595001.0,34486010.0,0.0,0.445244,0.060089,0.072940,0.070449,0.015529,0.335750,0.000000
2,1,8,101257782,41023229.0,5016478,13176666,11482239.0,5289744.0,23253517.0,0.0,0.405137,0.049542,0.130130,0.113396,0.052240,0.229647,0.000000


In [50]:
df_merged.head(3)

,estado,municipio,cve_inegi,id_estado,id_municipio,clasif,idh,gdh,ID_ENTIDAD,ID_MUNICIPIO,NOM_MUN
0,Aguascalientes,Asientos,1002,1,2,1025,0.665,MEDIO,1,2,Asientos
1,Aguascalientes,Cosío,1004,1,4,660,0.699,MEDIO,1,4,Cosío
2,Aguascalientes,San José de Gracia,1008,1,8,576,0.708,ALTO,1,8,San José de Gracia


In [51]:
# Unimos el dataframe df2 con df_merged (que contiene el idh) para generar el datafame final.
df_final = pd.merge(left=df_merged, right= df2, how = "right", left_on = ["id_estado", "id_municipio"], right_on = ["ID_ENTIDAD", "ID_MUNICIPIO"])

In [52]:
# Verificamos la union de los df's
df_final.head()

,estado,municipio,cve_inegi,id_estado,id_municipio,clasif,idh,gdh,ID_ENTIDAD_x,ID_MUNICIPIO_x,...,"Bienes muebles, inmuebles e intangibles",Inversión pública,Deuda pública,sp_porc,ms_porc,sg_porc,taoa_porc,bmii_porc,ip_porc,dp_porc
0,Aguascalientes,Asientos,1002,1,2,1025,0.665,MEDIO,1,2,...,120474.0,34970738.0,4003443.0,0.399935,0.062146,0.122725,0.085057,0.000789,0.228976,0.026213
1,Aguascalientes,Cosío,1004,1,4,660,0.699,MEDIO,1,4,...,1595001.0,34486010.0,0.0,0.445244,0.060089,0.072940,0.070449,0.015529,0.335750,0.000000
2,Aguascalientes,San José de Gracia,1008,1,8,576,0.708,ALTO,1,8,...,5289744.0,23253517.0,0.0,0.405137,0.049542,0.130130,0.113396,0.052240,0.229647,0.000000
3,Baja California,Ensenada,2001,2,1,211,0.755,ALTO,2,1,...,56198747.0,127376775.0,52528341.0,0.579578,0.090477,0.130388,0.056695,0.034005,0.077073,0.031784
4,Baja California,Tijuana,2004,2,4,90,0.786,ALTO,2,4,...,180836183.0,608300478.0,141658112.0,0.594838,0.086855,0.090333,0.065548,0.031556,0.106149,0.024720


In [53]:
# Eliminamos columnas duplicadas y otras que no se requieren
df_final.drop(["ID_ENTIDAD_x", "ID_MUNICIPIO_x"], axis = 1, inplace = True)
df_final.head()

,estado,municipio,cve_inegi,id_estado,id_municipio,clasif,idh,gdh,NOM_MUN,ID_ENTIDAD_y,...,"Bienes muebles, inmuebles e intangibles",Inversión pública,Deuda pública,sp_porc,ms_porc,sg_porc,taoa_porc,bmii_porc,ip_porc,dp_porc
0,Aguascalientes,Asientos,1002,1,2,1025,0.665,MEDIO,Asientos,1,...,120474.0,34970738.0,4003443.0,0.399935,0.062146,0.122725,0.085057,0.000789,0.228976,0.026213
1,Aguascalientes,Cosío,1004,1,4,660,0.699,MEDIO,Cosío,1,...,1595001.0,34486010.0,0.0,0.445244,0.060089,0.072940,0.070449,0.015529,0.335750,0.000000
2,Aguascalientes,San José de Gracia,1008,1,8,576,0.708,ALTO,San José de Gracia,1,...,5289744.0,23253517.0,0.0,0.405137,0.049542,0.130130,0.113396,0.052240,0.229647,0.000000
3,Baja California,Ensenada,2001,2,1,211,0.755,ALTO,Ensenada,2,...,56198747.0,127376775.0,52528341.0,0.579578,0.090477,0.130388,0.056695,0.034005,0.077073,0.031784
4,Baja California,Tijuana,2004,2,4,90,0.786,ALTO,Tijuana,2,...,180836183.0,608300478.0,141658112.0,0.594838,0.086855,0.090333,0.065548,0.031556,0.106149,0.024720


In [54]:
# Eliminamos otras columnas
df_final.drop(["NOM_MUN", "ID_ENTIDAD_y"], axis = 1, inplace = True)
df_final.head(3)

,estado,municipio,cve_inegi,id_estado,id_municipio,clasif,idh,gdh,ID_MUNICIPIO_y,Total de egresos,...,"Bienes muebles, inmuebles e intangibles",Inversión pública,Deuda pública,sp_porc,ms_porc,sg_porc,taoa_porc,bmii_porc,ip_porc,dp_porc
0,Aguascalientes,Asientos,1002,1,2,1025,0.665,MEDIO,2,152726780,...,120474.0,34970738.0,4003443.0,0.399935,0.062146,0.122725,0.085057,0.000789,0.228976,0.026213
1,Aguascalientes,Cosío,1004,1,4,660,0.699,MEDIO,4,102713281,...,1595001.0,34486010.0,0.0,0.445244,0.060089,0.072940,0.070449,0.015529,0.335750,0.000000
2,Aguascalientes,San José de Gracia,1008,1,8,576,0.708,ALTO,8,101257782,...,5289744.0,23253517.0,0.0,0.405137,0.049542,0.130130,0.113396,0.052240,0.229647,0.000000


In [55]:
# Las últimas columnas eliminadas
df_final.drop(["cve_inegi", "ID_MUNICIPIO_y"], axis = 1, inplace = True)
# El dataframe final
df_final.head()

,estado,municipio,id_estado,id_municipio,clasif,idh,gdh,Total de egresos,Servicios personales,Materiales y suministros,...,"Bienes muebles, inmuebles e intangibles",Inversión pública,Deuda pública,sp_porc,ms_porc,sg_porc,taoa_porc,bmii_porc,ip_porc,dp_porc
0,Aguascalientes,Asientos,1,2,1025,0.665,MEDIO,152726780,6.108078e+07,9491404,...,120474.0,34970738.0,4003443.0,0.399935,0.062146,0.122725,0.085057,0.000789,0.228976,0.026213
1,Aguascalientes,Cosío,1,4,660,0.699,MEDIO,102713281,4.573242e+07,6171900,...,1595001.0,34486010.0,0.0,0.445244,0.060089,0.072940,0.070449,0.015529,0.335750,0.000000
2,Aguascalientes,San José de Gracia,1,8,576,0.708,ALTO,101257782,4.102323e+07,5016478,...,5289744.0,23253517.0,0.0,0.405137,0.049542,0.130130,0.113396,0.052240,0.229647,0.000000
3,Baja California,Ensenada,2,1,211,0.755,ALTO,1652675878,9.578552e+08,149529041,...,56198747.0,127376775.0,52528341.0,0.579578,0.090477,0.130388,0.056695,0.034005,0.077073,0.031784
4,Baja California,Tijuana,2,4,90,0.786,ALTO,5730603903,3.408784e+09,497731034,...,180836183.0,608300478.0,141658112.0,0.594838,0.086855,0.090333,0.065548,0.031556,0.106149,0.024720


In [56]:
# Las últimas 5 filas del df final.
df_final.tail()

,estado,municipio,id_estado,id_municipio,clasif,idh,gdh,Total de egresos,Servicios personales,Materiales y suministros,...,"Bienes muebles, inmuebles e intangibles",Inversión pública,Deuda pública,sp_porc,ms_porc,sg_porc,taoa_porc,bmii_porc,ip_porc,dp_porc
415,Zacatecas,Mazapil,32,26,1517,0.625,MEDIO,163583623,34156009.0,26941219,...,1626309.0,32899993.0,0.0,0.208798,0.164694,0.217934,0.176501,0.009942,0.201120,0.000000
416,Zacatecas,Miguel Auza,32,29,1366,0.638,MEDIO,110602687,28808960.0,11041507,...,132055.0,51632060.0,0.0,0.260473,0.099830,0.119454,0.052224,0.001194,0.466825,0.000000
417,Zacatecas,Noria de Ángeles,32,35,1367,0.638,MEDIO,55378707,14987630.0,4367358,...,0.0,15867144.0,0.0,0.270639,0.078863,0.135583,0.077174,0.000000,0.286521,0.000000
418,Zacatecas,Susticacán,32,43,1739,0.605,MEDIO,12565324,4007752.0,903132,...,120525.0,4839207.0,0.0,0.318953,0.071875,0.182461,0.022916,0.009592,0.385124,0.000000
419,Zacatecas,Zacatecas,32,56,21,0.822,MUY ALTO,664028346,232533195.0,49198763,...,6139589.0,233361980.0,13701287.0,0.350186,0.074091,0.113400,0.081010,0.009246,0.351434,0.020634


In [57]:
# Estructura e información del df final.
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 0 to 419
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   estado                                                  420 non-null    object 
 1   municipio                                               420 non-null    object 
 2   id_estado                                               420 non-null    int64  
 3   id_municipio                                            420 non-null    int64  
 4   clasif                                                  420 non-null    object 
 5   idh                                                     420 non-null    object 
 6   gdh                                                     420 non-null    object 
 7   Total de egresos                                        420 non-null    int64  
 8   Servicios personales                    

Como producto final de este proyecto tenemos el dataset o conjunto de datos que aparece en las celdas anteriores.
El cual, como se ha mencionado, es el resultado de la **conjugación y transformacion de 3 conjuntos independientes**, el catálogo de municipios de México, de donde se seleccionó la muestra y los "id" de los municipios, el conjunto de finanzas municipales del año 2015 y el de IDH 2015.

El resultado es un conjunto de datos de 22 columnas y 420 filas (una por cada municipio).

Cada columna equivale a una variable a estudiar o comparar.

Entre las más sobresalientes tenemos: 

-Nombre del municipio y entidad, así como sus "id's" o claves.

-Clasificación o posición que guarda en orden númerico (respecto al idh)

-El IDH (indicador numérico) y clasificación categórica (alto, medio, bajo, etc.)

-Los montos del egreso total anual y por capítulo del gasto municipal

-Los porcentajes que representa cada capítulo del gasto respecto del egreso total.

Hay que anotar que este proyecto logra crear una tabla dinámica por cada municipio (realiza una transposición de categorías y valores) concatenarlas una encima de otra para generar un tabla de las finanzas públicas de 420 municipios.

El dataset o dataframe final, se encuentra listo para realizar exploración estadística, visualizaciones y transformaciones últimas para llevar a cabo el modelado.